Start with some boilerplate from Google so we can add EE layers to folium maps

In [6]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 10,
  'palette': ['000000', 'F5F5F5']}



In [4]:
# set up our SSEBOP runs for comparisons

import os

import ee
from ssebop_stats import ssebop, ee_manager

images = {}

for year in (2022, ):  # range(2015, 2023):
    runner = ssebop.SSEBOPer()
    runner.export_folder = f"year_comparison_{year}"
    runner.comparison(year, "03-01", "05-31", "06-01", "07-31", method="divide")
    images[year] = runner

CLASSIFIER_YEAR = 2019
    
runner_march_april = ssebop.SSEBOPer()
runner_march_april.pixel_reducer = "min"
runner_march_april.run(CLASSIFIER_YEAR, "03-01", "04-30")
runner_march_april.export()

runner_june_july = ssebop.SSEBOPer()
runner_june_july.pixel_reducer = "max"
runner_june_july.run(CLASSIFIER_YEAR, "06-01", "07-31")
runner_june_july.export()

In [28]:
params = dict(
    scale=30,
    maxPixels=1e12,
    fileNamePrefix="2022_comparison_from_jupyter_",
    fileDimensions=12800,
    crs='EPSG:3310',
    folder='comparison_2022_jupyter',
    description='comparison_2022_jupyter'
)

task = ee.batch.Export.image.toDrive(images[2022].results, **params)
task.start()

In [2]:
params = dict(
    scale=30,
    maxPixels=1e12,
    fileNamePrefix=f"{CLASSIFIER_YEAR}_spring_from_jupyter_",
    fileDimensions=12800,
    crs='EPSG:3310',
    folder=f'comparison_spring_{CLASSIFIER_YEAR}_jupyter',
    description=f'comparison_spring_{CLASSIFIER_YEAR}_jupyter'
)

task = ee.batch.Export.image.toDrive(runner_march_april.results, **params)
task.start()

In [3]:
params = dict(
    scale=30,
    maxPixels=1e12,
    fileNamePrefix=f"{CLASSIFIER_YEAR}_summer_from_jupyter_",
    fileDimensions=12800,
    crs='EPSG:3310',
    folder=f'comparison_{CLASSIFIER_YEAR}_jupyter',
    description=f'comparison_summer_{CLASSIFIER_YEAR}_jupyter'
)

task = ee.batch.Export.image.toDrive(runner_june_july.results, **params)
task.start()

In [4]:
# OK, let's try another approach
# 1 - a function to compare with subtraction
# 2 - then run classification
# 3 - zonal stats with majority (can we do this in EE if we're tiling? Not sure. May still need to download and do it)

SPRING_DOUBLE_DIFFERENCE_THRESHOLD = 0
SPRING_DOUBLE_RAW_ET_THRESHOLD = 2
DOUBLE_FALLOW_DIFFERENCE_THRESHOLD = 1.5
DOUBLE_FALLOW_RAW_ET_THRESHOLD = 3
CROPPED_LIKELY_THRESHOLD = 3
#           Class 0,       1 Sprg,  2 Dbl,    3 Fallow, 4 Likely, 5 Cropped
CLASS_COLORS = ["FFFFFF", "A3FF73", "A3FF73", "FFD37F", "0070FF", "0070FF"]

def simple_classifier(difference_img, raw_summer_et_img):
    """
    If difference is negative
        and summer ET less than 2:
            Class 1, Spring Crop, Summer Fallow
        else
            Class 2, Double Crop or Perennial
    Else if difference < 1.5
        if summer ET > 3
            class 2 Double Crop Perennial
        else
            Class 3, Fallow
    Else if difference < 3
        Class 4, Probably Cropped
    Else
        Class 5, Cropped
    """
    
    # OK, this is going to be funky and I feel like there's a better way, but my brain is mush.
    # We're going to separately get all the image classes with boolean logic, then multiply to turn
    # them into the correct class IDs. But there's going
    # to be some overlap (eg, checking if the difference is less than 1.5 for class 2 and 3),
    # so at the end, to combine, we'll use min() to combine, which takes earlier classes as higher priority.
    # It seems safest to *also* make sure to do the full logic chain for each one, but that's preeetty annoying.
    # could also just do .not() throughout the chain. Also not ideal, but duplicates less logic.
    
    difference_negative = difference_img.lt(SPRING_DOUBLE_DIFFERENCE_THRESHOLD)
    summer_lt_thresh = raw_summer_et_img.lt(SPRING_DOUBLE_RAW_ET_THRESHOLD)
    class_1_spring = difference_negative.And(summer_lt_thresh)
    
    class_2_option2 = difference_img.lt(DOUBLE_FALLOW_DIFFERENCE_THRESHOLD).And(raw_summer_et_img.lt(DOUBLE_FALLOW_RAW_ET_THRESHOLD))
    class_2_double_crop = difference_negative.And(summer_lt_thresh.Not()).Or(class_2_option2).multiply(2)  # make a boolean image, then multiply by 2 to get the class ID there
    
    class_3_fallow = (difference_img.gt(SPRING_DOUBLE_DIFFERENCE_THRESHOLD)
                        .And(difference_img.lt(DOUBLE_FALLOW_DIFFERENCE_THRESHOLD))
                        .And(raw_summer_et_img.lt(DOUBLE_FALLOW_RAW_ET_THRESHOLD))
                        .multiply(3))
    
    class_4_likely_cropped = (difference_img.gt(DOUBLE_FALLOW_DIFFERENCE_THRESHOLD)
                                .And(difference_img.lt(CROPPED_LIKELY_THRESHOLD))
                                .multiply(4))
    
    class_5_cropped = (difference_img.gt(CROPPED_LIKELY_THRESHOLD)
                        .multiply(5))
        
    class_img = (class_5_cropped
                 .max(class_2_double_crop)
                 .max(class_3_fallow)
                 .max(class_4_likely_cropped)
                 .max(class_1_spring))
    

    return class_img
    

In [22]:
# Create a folium map object.
my_map = folium.Map(location=[37.5, -120.5], zoom_start=8)

# Add the elevation model to the map object.
#my_map.add_ee_layer(images[2022].results, vis_params, 'GEE ET Ratio 2022')
#my_map.add_ee_layer(runner_march_april.results, vis_params, 'Spring 2022 Min ET')
#my_map.add_ee_layer(runner_june_july.results, vis_params, 'Summer 2022 Max ET')

comparison = runner_june_july.results.subtract(runner_march_april.results)
classified = simple_classifier(comparison, runner_june_july.results)
my_map.add_ee_layer(classified, {'min':0, 'max':5, 'palette': CLASS_COLORS}, "Classification")


# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [20]:
exporter = ee_manager.Image(classified)
exporter.export(**dict(
    scale=30,
    maxPixels=1e12,
    fileNamePrefix=f"classified_{CLASSIFIER_YEAR}",
    fileDimensions=12800,
    crs='EPSG:3310',
    folder=f"classified_{CLASSIFIER_YEAR}",
    description=f"classified_{CLASSIFIER_YEAR}"
))

In [ ]:
exporter.download()
exporter.mosaic()

In [14]:
ranges = (("03-01", "03-31", "march"), ("04-01", "04-30", "april"),
          ("05-01", "05-31", "may"), ("06-01", "06-30", "june"),
          ("07-01", "07-31", "july"), ("08-01", "08-31", "august"))

for year in (2022, 2021, 2019, 2018, 2020):
    for time_range in ranges:
        runner = ssebop.SSEBOPer()
        runner.pixel_reducer = "mean"
        runner.run(year, time_range[0], time_range[1])

        name = f"ssebop_et_{year}_{time_range[2]}_exported_2022_08_26"
        assetID = f"users/nrsantos/ssebop_et/{name}"
        print(assetID)
        ee_kwargs = {
            'description': name,
            'assetId': assetID,
            'scale': 30,
            'region': ee.FeatureCollection("projects/ucm-fallow-training/assets/central_valley_alluvial_boundary").geometry(),
            'maxPixels': 1e12,
            'shardSize': runner.tile_size,  # multiple of shardSize default 256. Should split into about 9 tiles
            'crs': runner.crs
        }
        
        runner.task = ee.batch.Export.image.toAsset(runner.results, **ee_kwargs)
        runner.task.start()
        print(runner.task)


users/nrsantos/ssebop_et/ssebop_et_2022_march_exported_2022_08_26
<Task FTKAK4TZTZZ2DMSWOD45WY74 EXPORT_IMAGE: ssebop_et_2022_march_exported_2022_08_26 (UNSUBMITTED)>
users/nrsantos/ssebop_et/ssebop_et_2022_april_exported_2022_08_26
<Task F524BUDPAVP3ZT2ZGYT5IQAZ EXPORT_IMAGE: ssebop_et_2022_april_exported_2022_08_26 (UNSUBMITTED)>
users/nrsantos/ssebop_et/ssebop_et_2022_may_exported_2022_08_26
<Task T63DQHYBWTXWCUIZDNICR6IS EXPORT_IMAGE: ssebop_et_2022_may_exported_2022_08_26 (UNSUBMITTED)>
users/nrsantos/ssebop_et/ssebop_et_2022_june_exported_2022_08_26
<Task YJOT3BHZQ662JLDJJFWFAZKY EXPORT_IMAGE: ssebop_et_2022_june_exported_2022_08_26 (UNSUBMITTED)>
users/nrsantos/ssebop_et/ssebop_et_2022_july_exported_2022_08_26
<Task FUHULEWSXP2VMXJLFBJCR26J EXPORT_IMAGE: ssebop_et_2022_july_exported_2022_08_26 (UNSUBMITTED)>
users/nrsantos/ssebop_et/ssebop_et_2022_august_exported_2022_08_26
<Task JSEI2OBBG5HUEJRV2WSA6725 EXPORT_IMAGE: ssebop_et_2022_august_exported_2022_08_26 (UNSUBMITTED)>
user